In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [12]:
text ="""들판에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [14]:
t = Tokenizer()
t.fit_on_texts([text])

In [15]:
t.word_index

{'말이': 1,
 '들판에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [16]:
vocabSize =len(t.word_index)+1 #11 (0~10 ->1~11) 12

In [20]:
# 트레이닝 데이터 생성
sequences=[]
for line in text.split("\n") :
    encoded =t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [23]:
maxLen = max(len(s) for s in sequences) #최대 길이 6
#입력크기를 동일하게 해주기 위해 패딩 필요

In [25]:
sequences= pad_sequences(sequences, maxlen= maxLen, padding='pre')

In [30]:
x= sequences[:,:-1] #마지막 열을 제외한 
y= sequences[:, -1]

array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11])

In [31]:
y=to_categorical(y, num_classes=vocabSize) #원핫

In [32]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [33]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [34]:
model = Sequential()
model.add(Embedding(vocabSize, 10, input_length=maxLen-1)) #x데이터 y데이터 분리해서 -1 
#임베딩벡터의 차원은 10
model.add(SimpleRNN(32))
model.add(Dense(vocabSize, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x,y, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 998us/step - loss: 2.4824 - accuracy: 0.1818
Epoch 2/200
1/1 [==============================] - 0s 0s/step - loss: 2.4705 - accuracy: 0.1818
Epoch 3/200
1/1 [==============================] - 0s 997us/step - loss: 2.4584 - accuracy: 0.1818
Epoch 4/200
1/1 [==============================] - 0s 997us/step - loss: 2.4461 - accuracy: 0.1818
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4334 - accuracy: 0.1818
Epoch 6/200
1/1 [==============================] - 0s 0s/step - loss: 2.4204 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 998us/step - loss: 2.4068 - accuracy: 0.3636
Epoch 8/200
1/1 [==============================] - 0s 997us/step - loss: 2.3927 - accuracy: 0.4545
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 2.3780 - accuracy: 0.4545
Epoch 10/200
1/1 [==============================] - 0s 996us/step - loss: 2.3625 - accuracy: 0.4545
Epoch 11/200
1/1 [=

In [42]:
def sentence_generation(model, t, cw, n):
    #문장 예측 생성
    initWord = cw #시작단어 저장
    sentence =''
    for _ in range(n):
        #4번 반복
        #_ : 변수가 없다. 
        # print(t.texts_to_sequences([cw]))
        encoded = t.texts_to_sequences([cw])[0]
        encoded =pad_sequences([encoded], maxlen=5, padding='pre')
        # print(encoded)
        result = model.predict_classes(encoded)
        # print(result)
        for word, index in t.word_index.items():
            if index==result:
                break
        cw=cw+" "+ word #"들판에 있는"
        sentence = sentence+" "+ word
    sentence=initWord+sentence
    return sentence

sentence_generation(model, t, '들판에', 4)

'들판에 있는 말이 뛰고 있다'